In [1]:
import pandas as pd
import numpy as np

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk import pos_tag
import string

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aminoacid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aminoacid/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aminoacid/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/aminoacid/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
df = pd.read_csv('PizzaReview.csv')

In [3]:
### Select top10 Chain Pizza Restaurants
shop_list=["Domino","Pizza hut","Little caesars","Papa John","California pizza kitchen",
           "Papa Murphy","Sbarro","Marco's pizza","Chuck E. cheese","Cici's pizza"]
tem1 = pd.DataFrame(columns=['business_id','name','city','state','stars_x','review_count',
                             'attributes','user_id','stars_y','text','date'])

for a in shop_list:
    tem = df[df['name'].str.contains(a, na=False, case=False)==True]
    tem['name']=a
    tem1 = pd.concat([tem1,tem],axis = 0)
   
PizzaChain = tem1.copy()
PizzaChain.shape[0]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


17942

In [4]:
#"Pizza hut"
df1 = PizzaChain[PizzaChain['name'] == "Pizza hut"]
#restName = df1.business_id.unique()
#restName

In [5]:
df1=df1[['business_id','stars_x','state','text','stars_y','review_count']]
df1=df1.rename(columns={'stars_x':'shop_star','stars_y':'user_star'})
df1.head()

,business_id,shop_star,state,text,user_star,review_count
2378,kqhHw_9KA30_RYk66mMbJA,1.0,BC,So now for an update.\n\nI've ordered 3x from ...,1.0,17
2379,kqhHw_9KA30_RYk66mMbJA,1.0,BC,"I came here for an interview, I found out that...",1.0,17
2380,kqhHw_9KA30_RYk66mMbJA,1.0,BC,"I know it's Pizza Hut, but even with my expect...",1.0,17
2381,kqhHw_9KA30_RYk66mMbJA,1.0,BC,Lunch time they have the $3.99 personal pan pi...,2.0,17
2382,kqhHw_9KA30_RYk66mMbJA,1.0,BC,This weekend I ordered delivery for the first ...,3.0,17


In [ ]:
#user_star
p3 = sns.countplot(x='user_star', data=df1) #so many 1.0 reviews

In [6]:
review = df1[['business_id','text','user_star','shop_star']]
review.head()

,business_id,text,user_star,shop_star
2378,kqhHw_9KA30_RYk66mMbJA,So now for an update.\n\nI've ordered 3x from ...,1.0,1.0
2379,kqhHw_9KA30_RYk66mMbJA,"I came here for an interview, I found out that...",1.0,1.0
2380,kqhHw_9KA30_RYk66mMbJA,"I know it's Pizza Hut, but even with my expect...",1.0,1.0
2381,kqhHw_9KA30_RYk66mMbJA,Lunch time they have the $3.99 personal pan pi...,2.0,1.0
2382,kqhHw_9KA30_RYk66mMbJA,This weekend I ordered delivery for the first ...,3.0,1.0


In [7]:
#pre-processing
def text_preprocess(text):
    text1 = [word for word in text if word not in string.punctuation] #punctuation
    text1 = ''.join(text1)
    
    text2 = text1.split() #split
    
    stop_words = stopwords.words('english')     #stop_words
    stop_words.extend(['pizza','good','food','great','one','get','back','would','really','also','got','u','love','ive','im',
                      "food","drink","eat", "now", "pizza", "flavor", "flavors", "Dominos", "Domino", "time","told","order","ordered","called",
                  "store", "said","one","pizzas","place","give","got","call","asked","even","really","Hut","say","will","still","ready",
                 "little","Papa","John", "Johns","Caesars","go","Murphy","Sbarro","Marco","Marcos","E","Chuck","Cici","always",
                 "hour","never","first","minute","phone","minutes","back","u","come","going","us","lot","son",
                      "hut",'location', 'customer', 'dont', 'like', 'didnt','could', 'ever', 'online', 
                       'take', 'guy', 'delivered', 'waiting', 'took','way','half','day','cant',
                    'came', 'ordering', 'tell', 'min', 'later','people','experience','person',
                     'year','people', 'person','thing','hold','delivery', 'service', 'manager', 'driver', 'cold', 'cheese', 
                       'hour', 'time', 'crust', 'employee', 'wait', 'wrong', 'star', 'last', 'pick', 'horrible', 
                       'credit', 'money', 'rude', 'bad', 'number', 'terrible', 'sauce', 
                       'night', 'refund', 'hung', 'review', 'business', 'wasnt', 'someone',
                       'hour', 'time', 'name', 'nothing', 'customer', 'employee', 'staff', 'restaurant', 'late', 'sure', 'min', 'box', 'order', 'card', 'work', 'door', 'extra', 'address', 'right', 'star', 'corporate', 'couldnt', 
                       'problem', 'something', 'pm', 'new', 'wont', 'ask', 'deliver', 'home'
                       ])
    #stop_words2 = stop_words
    text3 = [word.lower() for word in text2 if word.lower() not in stop_words]   #lower
    
    wnl = WordNetLemmatizer()    #lemmatization
    text4 = pos_tag([wnl.lemmatize(word) for word in text3])
    text5 = [word for word,type0 in text4 if type0 in ['NN']] #just keep noun
    return text5

In [ ]:
tem = text_preprocess('you are a cute pigs and dogs! hut, and far big small you are dogs good good good good good good')
tem



In [8]:
df3 = review.copy()
df3['text_processed']= df3['text'].apply(text_preprocess)

In [ ]:
df3.head()

In [9]:
#TF-IDF
from collections import Counter
import math

def tf(word, count):
    return count[word] / sum(count.values())
def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)
def idf(word, count_list):
    return math.log(len(count_list)) / (1 + n_containing(word, count_list))
def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)

def count_term(text):
    count = Counter(text)
    return count

In [ ]:
#concat reviews #group by bussiness_id
loop = df3[df3['user_star'] == 1]['text_processed'].count()
b = df3[df3['user_star'] == 1]
b.reset_index(drop=True, inplace=True)
wordlist = []
for i in range(0,loop-1): 
    for word in b['text_processed'][i]:
        wordlist.append(word)
text1 = wordlist
review_staris1 = text1

loop = df3[df3['user_star'] == 5]['text_processed'].count()
b = df3[df3['user_star'] == 5]
b.reset_index(drop=True, inplace=True)
wordlist = []
for i in range(0,loop-1): 
    for word in b['text_processed'][i]:
        wordlist.append(word)
text5 = wordlist
review_staris5 = text5



In [ ]:
texts = [text1,text5]
countlist = []
for text in texts:
    countlist.append(count_term(text))
for i, count in enumerate(countlist):
    print("Top words in review_starIs {}".format(i + 1))
    scores = {word: tfidf(word, count, countlist) for word in count}
    sorted_words = sorted(scores.items(), key = lambda x: x[1], reverse=True)
    word_list = []
    for word, score in sorted_words[:10]:
        #print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))
        #print("\"{}\"".format(word))
        word_list.append(word)
    print(word_list)  #top words in reviews


In [10]:
word_list = ['delivery','pay','card','credit','pickup','smile',
             'service','rude','polite','management','car','wait','refund','friendly','kind','manager','counter','staff','driver','employee',
             'cheese','fresh','pepperoni','sauce','star','delicious','thin','garlic','stick','breadstick','pie','pepper','crispy',
             'slice','email','pasta','wing','chicken','bread','steak','juice','yummy','mushroom','meat','marinara'
             'family','brownie', 'bbq', 'water','bacon','cinnamon','sausage'
            ]  

In [ ]:
word_list = ['delivery','service','rude','polite','management','car','wait','friendly','pepperoni',
             'sauce','wing','chicken','bread','meat','family'
            ]  

In [ ]:
# df_demo = pd.DataFrame(columns=(['c', 'a']+['aa']))
# df_demo.loc[df_demo.shape[0]] = dict(zip(df_demo.columns, ['dog',12,33]))
# df_demo
# ccc = 'cat'
# [ccc]+[12,33]

In [ ]:
# df_demo = pd.DataFrame(columns=(['c', 'a']+['aa']))
# df_demo.loc[0]=['cat', 3, 1]
# df_demo.loc[2]=['cat1', 31, 1]
# df_demo

# df.loc[df.shape[0]] = dict(zip(df.columns, value))

# a = a.append(pd.DataFrame(np.matrix(np.repeat(1, 11)),columns = a.columns))

# wordlist
# bussiness_list 
# bussiness_senti = 'bussi' + word_list
# for b in buss_list:
#     texttt = df3.text[df3.buss == b]
#     word_pos_score
#     v = [b] + word_pos_score
#     bussiness_senti.loc[df.shape[0]] = dict(zip(df.columns, value))
    

In [22]:
#bussiness sentiment
business_list = df3.business_id.unique()
word_list = ['wing','cheese','pepperoni','bread'] 
business_sentiment_score = pd.DataFrame(columns=(['business_id'] + word_list))
for b in business_list:
    #compute word positive score
    text0 = "".join(review for review in df3.text[df3.business_id == b]) 
    word_pos_score = []
    for word in word_list:
        review_set = [sentence + '.' for sentence in text0.split('.') if word in sentence]
        sia = SentimentIntensityAnalyzer()
        sentciciDeli = 0
        count = 0
        total = 0
        for sentence in  review_set:
            count = count + 1
            d = sia.polarity_scores(sentence)
            value = d['compound']
            total = total + value
        if count != 0:
            pos_score = float(total)/float(count)
        else :
            pos_score = None
        word_pos_score.append(pos_score)
    tem_busi_score = [b] + word_pos_score
    business_sentiment_score.loc[business_sentiment_score.shape[0]] = dict(zip(business_sentiment_score.columns, tem_busi_score))


In [40]:
business_sentiment_score
business_user_star = df3[['business_id','shop_star']]
result = pd.merge(left = business_sentiment_score, right = business_user_star, how = 'inner', on = ['business_id'])
result = result.drop_duplicates()
result.head()

,business_id,wing,cheese,pepperoni,bread,shop_star
0,kqhHw_9KA30_RYk66mMbJA,0,-0.30525,-0.0516,None,1.0
19,hWRZaOk4JWUPFhcomdOLwA,-0.31415,-0.1657,0.6452,0.3555,2.0
48,6EwA03Jcw5VaPc8BNiwrpw,0.109283,0.147008,0,-0.0875,2.0
79,HUEZLPIM64cDZnfHE1H3AA,0.00128667,0.33189,-0.24695,0.338888,3.5
139,_zy93iWV8Z-YmfvjWPcfAQ,0.0695571,-0.0878714,None,0.1158,2.0


In [24]:
np.mean(result,axis = 0)

wing         0.072730
cheese       0.025700
pepperoni    0.091862
bread        0.115951
shop_star    2.065517
dtype: float64

In [13]:
result.to_csv('words_sentiment_compound.csv')

In [ ]:
result.isnull().sum()/145

In [ ]:
#sentiment
text0 = "".join(review for review in df3.text[df3.user_star == 1])  #df3.text[df3.user_star == 1]
word_pos_score = []
for word in word_list:
    review_set = [sentence + '.' for sentence in text0.split('.') if word in sentence]
    sia = SentimentIntensityAnalyzer()
    sentciciDeli = 0
    count = 0
    total = 0
    for sentence in  review_set:
        count = count +1;
        d = sia.polarity_scores(sentence)
        value = d['pos']
        total = total + value;
    pos_score = float(total)/float(count)
    word_pos_score.append(pos_score) 

#merge word_list and word_pos_score
# print(word_list)
# print(word_pos_score)
word_sentiment = pd.concat([pd.DataFrame(word_list),pd.DataFrame(word_pos_score)], axis=1)
word_sentiment
    

In [ ]:
#sentiment
text0 = "".join(review for review in df3.text[df3.user_star == 5])  
word_pos_score = []
for word in word_list:
    review_set = [sentence + '.' for sentence in text0.split('.') if word in sentence]
    sia = SentimentIntensityAnalyzer()
    sentciciDeli = 0
    count = 0
    total = 0
    for sentence in  review_set:
        count = count +1;
        d = sia.polarity_scores(sentence)
        value = d['pos']
        total = total + value;
    pos_score = float(total)/float(count)
    word_pos_score.append(pos_score) 

#merge word_list and word_pos_score
# print(word_list)
# print(word_pos_score)
word_sentiment = pd.concat([pd.DataFrame(word_list),pd.DataFrame(word_pos_score)], axis=1)
word_sentiment